In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt


import numpy as np
from itertools import combinations
from scipy.stats import ttest_rel, wilcoxon
from scipy import stats
import re
import scikit_posthocs as sp 

from scipy.stats import shapiro, ttest_rel, wilcoxon
import pandas as pd
from itertools import combinations

## Extended Data Figure 2E

In [ ]:
CD44 = pd.read_excel('./CD44_Auszaehlungen.xlsx', sheet_name=0)


In [ ]:
CD44


In [ ]:
long = CD44.melt(
    value_vars=['decPAM','decPAM + EVT CM'],
    var_name='condition',
    value_name='value'
).dropna()

sns.set_theme(style="white")
fig, ax = plt.subplots(figsize=(3,3))

sns.boxplot(data=long, x='condition', y='value', showfliers=False, ax=ax, width=0.5)
sns.stripplot(data=long, x='condition', y='value', dodge=False, size=7, jitter=0.1,
              alpha=0.7, edgecolor='gray', linewidth=0.5, ax=ax)

ax.set_xlabel('')
ax.set_ylabel('CD44/CD14 colocalization FI count', fontsize=12)
ax.set_title('CD44/CD14 colocalization FI count', fontsize=12, pad=15)
ax.yaxis.set_tick_params(left=True, labelleft=True)
ax.xaxis.set_tick_params(bottom=True, top=False, labelbottom=True, labelsize=12)

plt.tight_layout()

plt.savefig('CD44_IF_stimulation.pdf')
plt.show()



from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu

cond1, cond2 = 'decPAM', 'decPAM + EVT CM'


x = CD44[cond1].dropna().astype(float)
y = CD44[cond2].dropna().astype(float)
n1, n2 = len(x), len(y)

def sh_test(a):
    if 3 <= len(a) <= 5000:
        stat, p = shapiro(a)
        return stat, p, (p > 0.05)
    return None, None, False


sh1, sh1_p, x_normal = sh_test(x)
sh2, sh2_p, y_normal = sh_test(y)

lev_stat, lev_p = (None, None)
eq_var = False
if n1 >= 2 and n2 >= 2:
    lev_stat, lev_p = levene(x, y, center='median')
    eq_var = (lev_p is not None) and (lev_p > 0.05)

if x_normal and y_normal:
    test_name = 'independent_t-test' if eq_var else 'Welch_t-test'
    stat, pval = ttest_ind(x, y, equal_var=eq_var)
else:
    test_name = 'Mann-Whitney_U'
    stat, pval = mannwhitneyu(x, y, alternative='two-sided', method='auto')

results_df = pd.DataFrame([{
    'group_1':          cond1,
    'group_2':          cond2,
    'n_group1':         n1,
    'n_group2':         n2,
    'shapiro_p_x':      sh1_p,
    'shapiro_p_y':      sh2_p,
    'both_normal':      x_normal and y_normal,
    'levene_p':         lev_p,
    'equal_var_assumed': eq_var,
    'test_used':        test_name,
    'test_statistic':   stat,
    'test_pvalue':      pval,
}])

print(results_df)



## Figure 3E

In [ ]:
macros = pd.read_excel('./Macro_Facs_data.xlsx')

In [ ]:
macros


In [ ]:
macros.dropna(subset=["actMac"], inplace=True)


In [ ]:
for macro in ['tiMac','inflaMac','trMac','actMac']:


    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=macros['tissue'].nunique())


    fig, ax = plt.subplots(figsize=(3, 3))


    sns.boxplot(
        data=macros,
        x='tissue',
        y=macro,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )


    sns.stripplot(
        data=macros,
        x='tissue',
        y=macro,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )


    handles, labels = ax.get_legend_handles_labels()
    n = macros['tissue'].nunique()

    ax.set_xlabel(macro, fontsize=12)
    ax.set_ylabel('Freq of total Mac', fontsize=12)
    ax.set_title(f'Freq {macro} of total Mac', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    ax.set_ylim(0,100)
    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,        
        labelbottom=True,  
        labelsize=12      
    )
    

    
    plt.tight_layout()

    plt.savefig(f'{macro}_new.pdf')
    plt.show()



    wide = macros.pivot(index='donor_id', columns='tissue', values=macro)


    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))


    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)

        
        diff = x - y

        
        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False  

        
        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)


## Extended Data Figure 4H

In [ ]:
tcell = pd.read_excel('./Tcell_rawdata.xlsx', sheet_name=0)


In [ ]:
tcell


In [ ]:
for cell in ['CD4+','CD8+','total']:


    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=tcell['tissue'].nunique())


    fig, ax = plt.subplots(figsize=(3, 3))


    sns.boxplot(
        data=tcell,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )


    sns.stripplot(
        data=tcell,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )


    handles, labels = ax.get_legend_handles_labels()
    n = tcell['tissue'].nunique()



    ax.set_xlabel(cell, fontsize=12)
    ax.set_ylabel('Freq of total CD45+', fontsize=12)
    ax.set_title(f'Freq {cell} of total CD45+', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    ax.set_ylim(0,15)
    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,         
        labelbottom=True,  
        labelsize=12       
    )

    plt.tight_layout()

    plt.savefig(f'{cell}.pdf')
    plt.show()

    wide = tcell.pivot(index='donor_id', columns='tissue', values=cell)

    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))

    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)

        diff = x - y

        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False  

        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)

## Extended Data Figure 4I

In [ ]:
granzymeB = pd.read_excel('./GranzymeB_ELISA_sum.xlsx')


In [ ]:
value = 'concentration'  

def plot_granzymeB(granzymeB, value):
    sns.set_theme(style="white")
    n_tissues = granzymeB['tissue'].nunique()
    palette = sns.color_palette(['#800000', '#005f87'], n_colors=n_tissues)

    fig, ax = plt.subplots(figsize=(3, 3))

    sns.boxplot(
        data=granzymeB, x='tissue', y=value, hue='tissue',
        palette=palette, showfliers=False, ax=ax, width=0.5
    )

    sns.stripplot(
        data=granzymeB, x='tissue', y=value, hue='tissue',
        palette=palette, dodge=False, size=7, jitter=0.1,
        alpha=0.7, edgecolor='gray', linewidth=0.5, ax=ax
    )

    ax.set_xlabel(value, fontsize=12) 
    ax.set_ylabel('GranzymeB concentration', fontsize=12)
    ax.set_title('GranzymeB concentration in explants', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    ax.xaxis.set_tick_params(which='both', bottom=True, top=False, labelbottom=True, labelsize=12)

    plt.tight_layout()
    plt.savefig('elisa_granzymeB.pdf')
    plt.show()


def paired_tests(granzymeB, value):
    wide = granzymeB.pivot(index='donor_id', columns='tissue', values=value)
    results = []

    for t1, t2 in combinations(wide.columns.tolist(), 2):
        pair = wide[[t1, t2]].dropna()
        x, y = pair[t1], pair[t2]
        n = len(pair)

        if 3 <= n <= 5000:
            _, sh_p = shapiro(x - y)
            normal = sh_p > 0.05
        else:
            sh_p, normal = (None, False)

        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1': t1,
            'tissue_2': t2,
            'n_pairs': n,
            'shapiro_p': sh_p,
            'normal_diff': normal,
            'test_used': test_name,
            'test_statistic': stat,
            'test_pvalue': pval,
        })

    return pd.DataFrame(results).sort_values('test_pvalue')


plot_granzymeB(granzymeB, value)
results_df = paired_tests(granzymeB, value)
print(results_df)

## Figure 5C

In [ ]:
nk = pd.read_excel('./NK_rawdata.xlsx', sheet_name=0)


In [ ]:
nk

In [ ]:
for nkcell in ['CD16+','CD39+','CD39- CD103+','CD39- CD103-']:


    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=nk['tissue'].nunique())


    fig, ax = plt.subplots(figsize=(3, 3))


    sns.boxplot(
        data=nk,
        x='tissue',
        y=nkcell,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )


    sns.stripplot(
        data=nk,
        x='tissue',
        y=nkcell,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )


    handles, labels = ax.get_legend_handles_labels()
    n = nk['tissue'].nunique()


    ax.set_xlabel(nkcell, fontsize=12)
    ax.set_ylabel('Freq of total NK', fontsize=12)
    ax.set_title(f'Freq {nkcell} of total NK', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    ax.set_ylim(0,100)
    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,         
        labelbottom=True, 
        labelsize=12       
    )


    plt.tight_layout()

    plt.savefig(f'{nkcell}.pdf')
    plt.show()



    wide = nk.pivot(index='donor_id', columns='tissue', values=nkcell)


    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))


    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)


        diff = x - y


        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False  


        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)


## Figure 5E

In [ ]:
dc = pd.read_excel('./DC_rawdata.xlsx', sheet_name=0)


In [ ]:
dc

In [ ]:

for cell in ['XCR1+','CD1c+','XCR1+ CD1c+']:


    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=dc['tissue'].nunique())


    fig, ax = plt.subplots(figsize=(3, 3))

    sns.boxplot(
        data=dc,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )

    sns.stripplot(
        data=dc,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )

    handles, labels = ax.get_legend_handles_labels()
    n = dc['tissue'].nunique()

    ax.set_xlabel(cell, fontsize=12)
    ax.set_ylabel('Freq of total CD45+', fontsize=12)
    ax.set_title(f'Freq {cell} of total CD45+', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    ax.set_ylim(0,2.5)
    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,         
        labelbottom=True,  
        labelsize=12       
    )

    plt.tight_layout()

    plt.savefig(f'{cell}.pdf')
    plt.show()

    wide = dc.pivot(index='donor_id', columns='tissue', values=cell)

    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))

    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)

        diff = x - y

        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False  

        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)

## Figure 5G

In [ ]:
pge2 = pd.read_excel('./PGE2_ELISA_sum.xlsx')


In [ ]:
pge2


In [ ]:
sns.set_theme(style="whitegrid")
palette = sns.color_palette("Set2", n_colors=pge2['tissue'].nunique())


fig, ax = plt.subplots(figsize=(12, 6))


sns.boxplot(
    data=pge2,
    x='method',
    y='concentration',
    hue='tissue',
    palette=palette,
    showfliers=False,
    ax=ax
)


sns.stripplot(
    data=pge2,
    x='method',
    y='concentration',
    hue='tissue',
    palette=palette,
    dodge=True,
    size=6,
    jitter=0.2,
    alpha=0.7,
    edgecolor='gray',
    linewidth=0.5,
    ax=ax
)


handles, labels = ax.get_legend_handles_labels()
n = pge2['tissue'].nunique()
ax.legend(handles[:n], labels[:n], title='Tissue',
          bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)


ax.set_xlabel('Method', fontsize=12)
ax.set_ylabel('Concentration', fontsize=12)
ax.set_title('Concentration by Method and Tissue', fontsize=14, pad=15)
plt.setp(ax.get_xticklabels(), rotation=30, ha='right')


plt.tight_layout()

plt.savefig('elisa_pge2.pdf')
plt.show()

In [ ]:
desc = (pge2
        .groupby(['method', 'tissue'])['concentration']
        .agg(median='median',
             q1=lambda x: x.quantile(.25),
             q3=lambda x: x.quantile(.75),
             n='count')
        .round(3)                           
        .reset_index())

print("\nMedians and IQRs")
print(desc)

In [ ]:
def donor_from_sample(s: str) -> str:
    
    return re.sub(r'^(DB|DP|EVT)\s+', '', s).strip()

pge2['donor'] = pge2['sample'].apply(donor_from_sample)


exp = pge2.query("method == 'explant' and tissue in ['decB','decP']")

exp_wide = (exp.pivot_table(index='donor',
                            columns='tissue',
                            values='concentration',
                            aggfunc='mean')
                .dropna())              

w_exp = stats.wilcoxon(exp_wide['decB'], exp_wide['decP'],
                       alternative='greater')

print(f"\nExplants decB vs decP  (paired on {len(exp_wide)} donors)")
print(f"Wilcoxon W = {w_exp.statistic:.0f}, p = {w_exp.pvalue:.4g}")

In [ ]:
iso_df = pge2.query("method == 'isolates' or tissue == 'EVT'")
groups = [g['concentration'].values
          for _, g in iso_df.groupby('tissue')]

kw_res = stats.kruskal(*groups)
print("\nCell isolates decB vs decP vs EVT")
print(f"Kruskal-Wallis H = {kw_res.statistic:.2f}, p = {kw_res.pvalue:.4g}")

# ---- post-hoc Dunn test with Holm correction ----
dunn = sp.posthoc_dunn(
            iso_df, val_col='concentration',
            group_col='tissue', p_adjust='holm')
print("\nDunn post-hoc (adjusted p-values)\n", dunn.round(4))

## Extended Figure 5B

In [ ]:
qPCR = pd.read_excel('./qPCR_rawdata.xlsx', sheet_name=0)


In [ ]:
qPCR


In [ ]:
for sample in ['PRL','IGFBP1']:

    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=qPCR['tissue'].nunique())

    fig, ax = plt.subplots(figsize=(3, 3))

    sns.boxplot(
        data=qPCR,
        x='tissue',
        y=sample,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )

    sns.stripplot(
        data=qPCR,
        x='tissue',
        y=sample,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )

    handles, labels = ax.get_legend_handles_labels()
    n = qPCR['tissue'].nunique()

    ax.set_xlabel(sample, fontsize=12)
    ax.set_ylabel('Relative mRNA expression', fontsize=12)
    ax.set_title(f'Relative mRNA expression of {sample}', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,         
        labelbottom=True,  
        labelsize=12       
    )

    plt.tight_layout()

    plt.savefig(f'{sample}.pdf')
    plt.show()


    wide = qPCR.pivot(index='donor_id', columns='tissue', values=sample)

    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))

    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)

        diff = x - y

        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False

        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)

## Extended Figure 5E

In [ ]:
CD82 = pd.read_excel('./CD82_rawdata.xlsx', sheet_name=0)


In [ ]:
CD82


In [ ]:
for cell in ['CD82']:

    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=CD82['tissue'].nunique())

    fig, ax = plt.subplots(figsize=(3, 3))


    sns.boxplot(
        data=CD82,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )

    sns.stripplot(
        data=CD82,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )

    handles, labels = ax.get_legend_handles_labels()
    n = CD82['tissue'].nunique()


    ax.set_xlabel(macro, fontsize=12)
    ax.set_ylabel('Freq of total Fib', fontsize=12)
    ax.set_title(f'Freq {cell} of total Fib', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)
    #ax.set_ylim(0,15)
    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,         
        labelbottom=True,  
        labelsize=12       
    )


    plt.tight_layout()

    plt.savefig(f'{cell}.pdf')
    plt.show()


    wide = CD82.pivot(index='donor_id', columns='tissue', values=cell)


    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))

    
    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)


        diff = x - y


        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False  

        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'

            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)

## Figure 6C

In [ ]:
fsc = pd.read_excel('/Users/alackner/Library/CloudStorage/OneDrive-Stanford/Documents/Ongoing-Projects/CITE-seq/manuscript/figures_2025/FACS/Fibroblast_FSC_total.xlsx', sheet_name=0)


In [ ]:
fsc


In [ ]:
for cell in ['FSC']:

    sns.set_theme(style="white")
    palette = sns.color_palette(['#800000','#005f87'], n_colors=fsc['tissue'].nunique())

    fig, ax = plt.subplots(figsize=(3, 3))

    sns.boxplot(
        data=fsc,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        showfliers=False,
        ax=ax,
        width=0.5
    )

    sns.stripplot(
        data=fsc,
        x='tissue',
        y=cell,
        hue='tissue',
        palette=palette,
        dodge=False,
        size=7,
        jitter=0.1,
        alpha=0.7,
        edgecolor='gray',
        linewidth=0.5,
        ax=ax
    )

    handles, labels = ax.get_legend_handles_labels()
    n = fsc['tissue'].nunique()


    ax.set_xlabel(cell, fontsize=12)
    ax.set_ylabel('FSC-A', fontsize=12)
    ax.set_title(f'{cell}', fontsize=12, pad=15)
    ax.yaxis.set_tick_params(left=True, labelleft=True)

    ax.xaxis.set_tick_params(
        which='both',      
        bottom=True,       
        top=False,         
        labelbottom=True,  
        labelsize=12       
    )


    plt.tight_layout()

    plt.savefig(f'{cell}.pdf')
    plt.show()

    wide = fsc.pivot(index='donor_id', columns='tissue', values=cell)

    tissues = wide.columns.tolist()
    pairs   = list(combinations(tissues, 2))


    results = []
    for t1, t2 in pairs:
        pair_df = wide[[t1, t2]].dropna()
        x = pair_df[t1]
        y = pair_df[t2]
        n = len(pair_df)

        diff = x - y

        if n >= 3 and n <= 5000:
            sh_stat, sh_p = shapiro(diff)
            normal = (sh_p > 0.05)
        else:
            sh_stat, sh_p = (None, None)
            normal = False  

        if normal:
            test_name = 'paired_t-test'
            stat, pval = ttest_rel(x, y, nan_policy='omit')
        else:
            test_name = 'wilcoxon'
            try:
                stat, pval = wilcoxon(x, y)
            except ValueError:
                stat, pval = (None, None)

        results.append({
            'tissue_1':        t1,
            'tissue_2':        t2,
            'n_pairs':         n,
            'shapiro_p':       sh_p,
            'normal_diff':     normal,
            'test_used':       test_name,
            'test_statistic':  stat,
            'test_pvalue':     pval,
        })

    results_df = pd.DataFrame(results).sort_values('test_pvalue')
    print(results_df)

## Figure 6E

In [ ]:
cortisol = pd.read_excel('./Cortisol_ELISA_sum.xlsx')


In [ ]:
sns.set_theme(style="whitegrid")
palette = sns.color_palette("Set2", n_colors=cortisol['tissue'].nunique())


fig, ax = plt.subplots(figsize=(12, 6))


sns.boxplot(
    data=cortisol,
    x='method',
    y='concentration',
    hue='tissue',
    palette=palette,
    showfliers=False,
    ax=ax
)

sns.stripplot(
    data=cortisol,
    x='method',
    y='concentration',
    hue='tissue',
    palette=palette,
    dodge=True,
    size=6,
    jitter=0.2,
    alpha=0.7,
    edgecolor='gray',
    linewidth=0.5,
    ax=ax
)


handles, labels = ax.get_legend_handles_labels()
n = cortisol['tissue'].nunique()
ax.legend(handles[:n], labels[:n], title='Tissue',
          bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)


ax.set_xlabel('Method', fontsize=12)
ax.set_ylabel('Concentration', fontsize=12)
ax.set_title('Concentration by Method and Tissue', fontsize=14, pad=15)
plt.setp(ax.get_xticklabels(), rotation=30, ha='right')


plt.tight_layout()

plt.savefig('elisa_cortisol.pdf')
plt.show()

In [ ]:
desc = (cortisol
        .groupby(['method', 'tissue'])['concentration']
        .agg(median='median',
             q1=lambda x: x.quantile(.25),
             q3=lambda x: x.quantile(.75),
             n='count')
        .round(3)                           
        .reset_index())

print("\nMedians and IQRs")
print(desc)
def donor_from_sample(s: str) -> str:
    
    return re.sub(r'^(DB|DP|EVT)\s+', '', s).strip()

cortisol['donor'] = cortisol['sample'].apply(donor_from_sample)


exp = cortisol.query("method == 'explant' and tissue in ['decB','decP']")

exp_wide = (exp.pivot_table(index='donor',
                            columns='tissue',
                            values='concentration',
                            aggfunc='mean')
                .dropna())              

w_exp = stats.wilcoxon(exp_wide['decB'], exp_wide['decP'],
                       alternative='greater')

print(f"\nExplants decB vs decP  (paired on {len(exp_wide)} donors)")
print(f"Wilcoxon W = {w_exp.statistic:.0f}, p = {w_exp.pvalue:.4g}")



iso_df = cortisol.query("method == 'isolates' or tissue == 'EVT'")
groups = [g['concentration'].values
          for _, g in iso_df.groupby('tissue')]

kw_res = stats.kruskal(*groups)
print("\nCell isolates decB vs decP vs EVT")
print(f"Kruskal-Wallis H = {kw_res.statistic:.2f}, p = {kw_res.pvalue:.4g}")


dunn = sp.posthoc_dunn(
            iso_df, val_col='concentration',
            group_col='tissue', p_adjust='holm')
print("\nDunn post-hoc (adjusted p-values)\n", dunn.round(4))